In [1]:
!pip install datasets pandas transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 20.4 MB/s 
     |████████████████████████████████| 5.3 MB 65.3 MB/s 
     |████████████████████████████████| 115 kB 61.1 MB/s 
     |████████████████████████████████| 212 kB 58.5 MB/s 
     |████████████████████████████████| 163 kB 46.5 MB/s 
     |████████████████████████████████| 127 kB 71.7 MB/s 
     |████████████████████████████████| 7.6 MB 60.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [3]:
!git config --global credential.helper store

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [5]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16


In [6]:
from datasets import load_dataset 
dataset = load_dataset("pythonist/PubMedQA", use_auth_token=True, encoding='ISO-8859–1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/pythonist___csv/pythonist--PubMedQA-75686b8f5aa447c8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 333
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 110
    })
})

In [9]:
# show_random_elements(datasets["train"])


In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [23]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        import ast
        answer = ast.literal_eval(answer)
        start_char = answer["answer_start"][0]
        # print(start_char)
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [25]:
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

---
# Fine-Tuning 

In [26]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [27]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-PubmedQA",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    push_to_hub=True,
)

In [28]:
from transformers import default_data_collator

data_collator = default_data_collator

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/pythonist/distilbert-base-uncased-finetuned-PubmedQA into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.31k/253M [00:00<?, ?B/s]

Download file runs/Oct17_04-39-10_93de6e6db13f/1665981565.3333683/events.out.tfevents.1665981565.93de6e6db13f.…

Download file training_args.bin:  45%|####5     | 1.53k/3.36k [00:00<?, ?B/s]

Download file runs/Oct17_04-39-10_93de6e6db13f/events.out.tfevents.1665981565.93de6e6db13f.69.0:  48%|####7   …

Clean file runs/Oct17_04-39-10_93de6e6db13f/1665981565.3333683/events.out.tfevents.1665981565.93de6e6db13f.69.…

Clean file training_args.bin:  30%|##9       | 1.00k/3.36k [00:00<?, ?B/s]

Clean file runs/Oct17_04-39-10_93de6e6db13f/events.out.tfevents.1665981565.93de6e6db13f.69.0:   6%|5         |…

Clean file pytorch_model.bin:   0%|          | 1.00k/253M [00:00<?, ?B/s]

In [30]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 333
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 420


Epoch,Training Loss,Validation Loss
1,No log,3.861109
2,No log,3.371441
3,No log,3.300969
4,No log,3.271797
5,No log,3.233702
6,No log,3.226179
7,No log,3.243842
8,No log,3.234160
9,No log,3.311186
10,No log,3.378516


***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evaluation *****
  Num examples = 110
  Batch size = 16
***** Running Evalua

TrainOutput(global_step=420, training_loss=2.2378494989304314, metrics={'train_runtime': 13021.2303, 'train_samples_per_second': 0.511, 'train_steps_per_second': 0.032, 'total_flos': 652612234659840.0, 'train_loss': 2.2378494989304314, 'epoch': 20.0})

In [31]:
trainer.save_model("pubmed-qa")


Saving model checkpoint to pubmed-qa
Configuration saved in pubmed-qa/config.json
Model weights saved in pubmed-qa/pytorch_model.bin
tokenizer config file saved in pubmed-qa/tokenizer_config.json
Special tokens file saved in pubmed-qa/special_tokens_map.json
Saving model checkpoint to distilbert-base-uncased-finetuned-PubmedQA
Configuration saved in distilbert-base-uncased-finetuned-PubmedQA/config.json
Model weights saved in distilbert-base-uncased-finetuned-PubmedQA/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-PubmedQA/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-PubmedQA/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.34k/253M [00:00<?, ?B/s]

Upload file runs/Oct17_08-52-20_b7e4cbdee051/events.out.tfevents.1665996843.b7e4cbdee051.50.0:  36%|###6      …

Upload file runs/Oct17_08-52-20_b7e4cbdee051/1665996843.5047455/events.out.tfevents.1665996843.b7e4cbdee051.50…

Upload file training_args.bin:  99%|#########9| 3.34k/3.36k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/pythonist/distilbert-base-uncased-finetuned-PubmedQA
   96c8426..e8acf45  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/pythonist/distilbert-base-uncased-finetuned-PubmedQA
   96c8426..e8acf45  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/pythonist/distilbert-base-uncased-finetuned-PubmedQA
   e8acf45..767521e  main -> main

   e8acf45..767521e  main -> main



---
# Evaluation

In [32]:
import torch
from tqdm import tqdm

acc = []

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

for batch in tqdm(trainer.get_eval_dataloader()):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
acc = sum(acc)/len(acc)

100%|██████████| 7/7 [01:01<00:00,  8.85s/it]


In [33]:
acc

0.5261479616165161

In [34]:
start_true

tensor([ 34, 343, 360,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 223])

In [35]:
start_pred

tensor([65,  0, 52,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

# Exact Match

In [36]:
((start_pred == start_true).sum()/len(start_pred)).item()

0.7142857313156128

In [37]:
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
# batch.keys()

In [40]:
print("T/F\tstart\tend\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
          f"pred\t{start_pred[i]}\t{end_pred[i]}\n")

T/F	start	end

true	34	35
pred	65	65

true	343	344
pred	0	0

true	360	372
pred	52	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	0	0
pred	0	0

true	223	231
pred	0	0



In [41]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-PubmedQA
Configuration saved in distilbert-base-uncased-finetuned-PubmedQA/config.json
Model weights saved in distilbert-base-uncased-finetuned-PubmedQA/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-PubmedQA/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-PubmedQA/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
